# EC-KCDSA

Elliptic Curve Korean Certificate-based Digital Signature Algorithm (KCDSA)

In [1]:
import random
import hashlib

In [2]:
# y^2 = x^3 + a*x + b

In [3]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
    
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
    
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
    
    is_on_curve((x3, y3), p)
        
    return x3, y3

def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
    
def apply_double_and_add_method(G, k, p):
    target_point = G
    
    k_binary = bin(k)[2:] #0b1111111001
    
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
        
        # doubling - always
        target_point = add_points(target_point, target_point, p)
        
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
    
    is_on_curve(target_point, p)
    
    return target_point

In [4]:
# Secp256k1
a = 0; b = 7
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [5]:
is_on_curve(G, p)

# Signing

In [6]:
message = "fenerbahce"

In [7]:
def find_hash(m):
    
    if isinstance(m, int):
        m = str(m)
    
    # to bytes
    m = str.encode(m)
        
    hash_value = hashlib.sha256(m).digest()
    # Convert the hash value to an integer
    return int.from_bytes(hash_value, 'big')

In [8]:
# Alice's private key
d = random.getrandbits(256)

# public key of Alice
Qa = apply_double_and_add_method(G = G, k = pow(d, -1, n), p = p)

In [9]:
# random key
k = random.getrandbits(256)
K = apply_double_and_add_method(G = G, k = k, p = p)

In [10]:
r = find_hash(K[0])

In [11]:
e = find_hash(message)

In [12]:
# TODO: hash should return bits and i should add two binary numbers
w = ( r + e ) % n

In [13]:
s = ( d * (k - w) ) % n

In [14]:
assert s != 0

In [15]:
# send (r, s) pair to Bob

# Verification

In [16]:
r, s

(2627115786895751047619043427573784819443544467753686718215604549245316874562,
 46991288793321660032041704358548977753200924831685655513531834284462795112390)

In [17]:
e = find_hash(message)

In [18]:
w = ( r + e ) % n

In [19]:
x = add_points(
    P = apply_double_and_add_method(G = Qa, k = s, p = p),
    Q = apply_double_and_add_method(G = G, k = w, p = p), 
    p = p
)

In [20]:
assert find_hash(x[0]) == r

# Proof

how signature s was calculated by Alice

s = d * (k - w)  mod n

s * d^-1 = k - w mod n

k = s * d^-1 + w

how bob calculated x

x = sQ + wG

x = s*d^1xG + wG

x = (s*d^-1 + w) x G

x = k x G

x = K

Alice found the signature r as the x coordinate of point K

r = hash(K[0])

this must be true always!

hash(x[0]) == r